# Background Maps

DATE: 11 June 2020, 18:00 - 21:00 UTC

AUDIENCE: Intermediate

INSTRUCTOR: Martin Bentley, Digital Geoscientist, [Agile](https://agilescientific.com/)

It is often useful to add a background map to a given dataset. There are a few tools that can do this, such as [`folium`](https://python-visualization.github.io/folium/) and [`ipyleaflet`](https://ipyleaflet.readthedocs.io/en/latest/) which build directly on the `leaflet` library for JavaScript. However, for a simple approach, we will stick to [`contextily`](https://github.com/darribas/contextily). There are some additional capabilities that we will not go over in this, but it should be enough to get started with.

Note: the inital load of tiles can take quite a while as they get downloaded. Subsequent loads of the same tiles will be much quicker as they cache locally.

This notebook is intended more as a demonstration of using contextily to add basemaps in a simple way.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)

First we will get some point data, in this case mines in Tanzania. Geopandas can download the file and import it directly from the source at [Geological and Mineral Information System](https://www.gmis-tanzania.com/) by the Geological Survey of Tanzania. If this download does not work, it is in the repo as `data/tanzania_mines.zip`.

In [ ]:
fname = 'https://www.gmis-tanzania.com/download/mines.zip'
mines = gpd.read_file(fname)

In [ ]:
mines

This can easily be plotted, as we have already done.

In [ ]:
mines.plot(column='miningexpl', legend=True)

In order to gain more context, we can plot this over a basemap of some kind. By default, `contextily` uses the Stamen Terrain tiles.

In [ ]:
base = mines.plot(column='miningexpl', legend=True)
ctx.add_basemap(base, crs=mines.crs)

Something worth noting is that the basemap is easily projected by giving it the `mines.crs` as a parameter. This is needed since the dataset using a local projected CRS, but we can largely ignore it.

In [ ]:
mines.crs

We can switch this to use lat-lon easily enough, by reprojecting the data to something like the WGS84 datum first.

In [ ]:
mines_deg = mines.to_crs(epsg=4326)
base = mines_deg.plot(column='miningexpl', legend=True, alpha=0.75)
ctx.add_basemap(base, crs=mines_deg.crs)

## Changing the Basemap

The leaflet providers are available in contextily, which allows for a variety of different styles and looks.

In [ ]:
ctx.providers.keys()

Many of these may have specific styles, which will look different. Compare the default [Mapnik style](https://www.openstreetmap.org/search?query=Kinshasa#map=13/-4.3385/15.3131) to the [Transport style](https://www.openstreetmap.org/search?query=Kinshasa#map=13/-4.3385/15.3131&layers=T), for example. Some of these styles may require API keys to use, and many will have usage limits of some kind.

In [ ]:
print(ctx.providers.OpenStreetMap.keys())
print(ctx.providers.Thunderforest.keys())
print(ctx.providers.Esri.keys())

Changing the basemap to use one of these is as simple as changing the `source` parameter. We can also make it fade a little by setting the `alpha` parameter.

In [ ]:
fig, ax = plt.subplots()
mines_deg.plot(ax=ax, column='miningexpl', legend=True, alpha=0.75)
ctx.add_basemap(ax, crs=mines_deg.crs,
               source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)

It is also possible to load custom tilemaps, if they support the standard XYZ format. This is useful if you have created one using your own data somewhere. We will use the tiling server hosted by the government of New South Wales.

It is also possible to request tiles based on an extent, which needs to be either in WGS84 (EPSG 4326) or Pseudo-Mercator (EPSG 3587).

In [ ]:
src = 'http://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_Base_Map/MapServer/tile/{z}/{y}/{x}'
west, south, east, north = 16783076.1, -4041012.6, 16851459.8, -3988135.3

The `bounds2img` method will download the tiles within a given bounding box as a three band array. The `ll` parameter is whether your data is in lon-lat or Pseudo-Mercator.

In [ ]:
sydney_img, sydney_ext = ctx.bounds2img(west, south, east, north,
                                       source=src, ll=False, zoom=10)
print(sydney_img.shape)
plt.imshow(sydney_img, extent=sydney_ext)

## Downloading Basemaps

While basemaps downloaded are cached locally (try re-running one of the above cells; it should be much quicker), sometimes we may want to download them to use elsewhere or to save the bandwidth. Contextily can do that easily.

Contextily sets a default zoom based on the extents, but we can change that if we want or need to. Higher zoom levels means downloading more tiles, but with higher resolution.

In [ ]:
ctx.howmany(west, south, east, north, 7, ll=False)

In [ ]:
ctx.howmany(west, south, east, north, 10, ll=False)

In [ ]:
ctx.howmany(west, south, east, north, 12, ll=False)

These will look different, because they are optimised to be viewed at a different zoom level. These changes include which features are shown, smoothing of lines, size of labels, and so on. The zoom levels for OpenStreetMap can be seen [here](https://wiki.openstreetmap.org/wiki/Zoom_levels). Most providers will be very similar if not the same.

Note: do not try and download large areas at high resolution unless absolutely necessary. Many providers will cut off access for excessive use, or may have a limited number of requests for a given service tier.

The following two blocks illustrate the difference in number of tiles downloaded at given zoom levels. Given the limitations of the browser, they might be clearer by looking at the downloaded files instead.

In [ ]:
sydney_10_img, sydney_10_ext = ctx.bounds2raster(west,
                             south,
                             east,
                             north,
                             "sydney_z10.tif",
                             source=src,
                             ll=False,
                             zoom=10
                            )
plt.imshow(sydney_10_img, extent=sydney_10_ext)

In [ ]:
sydney_12_img, sydney_12_ext = ctx.bounds2raster(west,
                             south,
                             east,
                             north,
                             "sydney_z12.tif",
                             source=src,
                             ll=False,
                             zoom=12
                            )
plt.imshow(sydney_12_img, extent=sydney_12_ext)

We can load a saved tiff using something like `rasterio`, or in ArcGIS/QGIS.

In [ ]:
import numpy as np
import rasterio as rio
from rasterio.plot import show as rioshow

with rio.open("sydney_z10.tif") as r:
    rioshow(r)

## Geocoding in `Contextily`

A really nice feature to have is being able to download basemaps given a placename. This is made very simple in contextily, through use of the `pygeo` geocoder. The places can be countries, cities, or other places.

In [ ]:
paraguay = ctx.Place('Paraguay', source=ctx.providers.Esri.DeLorme)
paraguay

This can be used as a basemap with existing data as already shown.

In [ ]:
rivers = gpd.read_file('zip://../data/ne_10m_rivers_lake_centerlines_trimmed.zip')
#rivers_clipped = rivers[rivers.intersects(paraguay.bbox)]
base = rivers.plot(color='red')
ctx.plot_map(paraguay, ax=base, axis_off=False)

In [ ]:
ctx.Place('Geneva', source=ctx.providers.CartoDB.Positron).plot()

In [ ]:
ctx.Place('Union Buildings', source=ctx.providers.Wikimedia, zoom=19).plot()

<hr />
<img src="https://avatars1.githubusercontent.com/u/1692321?v=3&s=200" style="float:center" width="40px" />
<p><center>Â© 2020 <a href="http://www.agilegeoscience.com/">Agile Geoscience</a> â€” <a href="https://creativecommons.org/licenses/by/4.0/">CC-BY</a></center></p>